In [1]:
import torch_geometric.datasets as datasets

mutag = datasets.TUDataset(root='data', name='AIDS')

In [5]:
from gnnexplain.nn.gcn import GraphGCN
from lightning import LightningModule

model = GraphGCN.load_from_checkpoint('checkpoints/MUTAGsplit=0-step=10-val_acc=60.00%.ckpt').cpu()
isinstance(model, LightningModule)

TypeError: GraphGCN.__init__() missing 1 required positional argument: 'dataset'

In [7]:
from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch

loader = DataLoader(mutag, batch_size=40000, shuffle=False)
batch = loader.__iter__().__next__().cpu()
l = batch.to_data_list()
train, test = Batch.from_data_list(l[:150]), Batch.from_data_list(l[150:])

In [8]:
(model(test).argmax(1) == test.y).float().mean()

tensor(1.)

In [12]:
from gnnexplain.model.gtree import *
import optuna

optuna.logging.set_verbosity(optuna.logging.WARNING)
opt = Optimizer(n_trials=1e3, max_ccp_alpha=1e-2, max_depth=10, lmb=1e-3)
expl = opt.optimize(train, model)

Best trial: 632. Best value: 0.863: 100%|██████████| 1000/1000.0 [03:13<00:00,  5.17it/s]


In [14]:
expl.accuracy(train)

0.84

In [15]:
expl.accuracy(test)

0.7631578947368421

In [17]:
n_val = len(mutag) // 10

In [29]:
val_data = mutag[1 * n_val : (1 + 1) * n_val]
train_data = mutag[:1 * n_val] + mutag[(1 + 1) * n_val:]

In [30]:
val_batch = Batch.from_data_list(val_data)
train_batch = Batch.from_data_list(train_data)

In [32]:
val_batch.x.shape

torch.Size([345, 7])

In [33]:
train_batch.x.shape

torch.Size([3026, 7])